# Using RAG Pattern with Weaviate and Azure Open AI

In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
import json
import requests
import weaviate
from weaviate import EmbeddedOptions
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import MetadataQuery, HybridVector, Move
import warnings

warnings.filterwarnings('ignore')

# Load the environment variables
load_dotenv()

aoai_key=os.environ['AZURE_OPENAI_API_KEY']
aoai_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
aoai_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
aoai_embedding=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
aoai_embedding_3=os.environ['AZURE_OPENAI_EMBEDDINGS_3_DEPLOYMENT']

## Utility Funtions

In [2]:
# JSON print beautifier
def json_print(data):
    print(json.dumps(data, indent=2))

# Print response beautifier
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)

## Create or connecto a Weaviate Embedded DB

* https://weaviate.io/developers/weaviate/connections/connect-embedded
* https://weaviate.io/developers/weaviate/connections/connect-local

In [ ]:
try:
    response = requests.get("http://localhost:8079/v1/schema")
    r_err = False
except Exception as err:    
    r_err = True

if r_err:
    client = weaviate.connect_to_embedded(
        version="1.26.1",  # e.g. version="1.26.5"
        headers={
            "X-OpenAI-BaseURL": aoai_endpoint,
            "X-Azure-Api-Key": aoai_key
        },
    )
    print("Connected to new instance")
elif response.status_code == 200:
    client = weaviate.connect_to_local(
        port=8079, 
        grpc_port=50050,    
        headers={
            "X-OpenAI-BaseURL": aoai_endpoint,
            "X-Azure-Api-Key": aoai_key
        }
    )
    print("Connected to existing instance")

print(client.is_ready())

In [ ]:
# Show Weaviate DB metadata
json_print(client.get_meta())

## Create a new Collection (EU Destinations)

https://weaviate.io/developers/weaviate/manage-data/collections

In [5]:
if client.collections.exists("eudestinations"):
    client.collections.delete("eudestinations")

* Using Azure OpenAI Embeddings: https://weaviate.io/developers/weaviate/model-providers/openai-azure/embeddings
* Usong Azure OpenAI: https://weaviate.io/developers/weaviate/model-providers/openai-azure/generative

In [ ]:
# Using a Weaviate collection with text-embedding-ada-002 and gpt-4-turbo-2024-04-09
client.collections.create(
    "eudestinations",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="title_vector",            
            resource_name="aoai-airlift-1",
            deployment_id=aoai_embedding,
            base_url=aoai_endpoint
        )
    ],
    generative_config=Configure.Generative.azure_openai(
        resource_name="aoai-airlift-1",
        deployment_id=aoai_deployment,
        base_url=aoai_endpoint
        # frequency_penalty=0,
        # max_tokens=500,
        # presence_penalty=0,
        # temperature=0.7,
        # top_p=0.7
    )    
)

## Read Europe Touristic Destinations

_Source: https://www.kaggle.com/datasets/faizadani/european-tour-destinations-dataset_

In [ ]:
file_path = '../data/eu_destinations_n.json'

with open(file_path, "r") as file:
    data = file.read()

ds = json.loads(data)
json_print(ds[0])

## Loading Embeddings into the Weaviate DB

https://weaviate.io/developers/weaviate/manage-data/import

In [8]:
collection = client.collections.get("eudestinations")

with collection.batch.dynamic() as batch:
    for i, d in enumerate(ds):
        weaviate_obj = {
            "destination": d["Destination"],
            "region": d["Region"],
            "country": d["Country"],
            "category": d["Category"],
            "annualtourists": d["Approximate Annual Tourists"],
            "foods": d["Famous Foods"],
            "language": d["Language"],
            "besttimevisit": d["Best Time to Visit"],
            "costliving": d["Cost of Living"],
            "cultural": d["Cultural Significance"],
            "description": d["Description"]
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )

## Query a specific Collection Dataset

In [9]:
collection = client.collections.get("eudestinations")

## Search Patterns

https://weaviate.io/developers/weaviate/search/basics

In [ ]:
# Fetch a DB object in ascending UUID order
response = collection.query.fetch_objects(
    limit=1
)
for o in response.objects:
    json_print(o.properties)

### RAG Search with Azure OpenAI

Retrieval Augmented Generation (RAG) is an architecture that augments the capabilities of a Large Language Model (LLM) like ChatGPT by adding an information retrieval system that provides grounding data. 

### Single Prompt Search
* Single prompt search **returns a generated response for each object** in the query results.
* Define object properties – using **{prop-name}** syntax – to interpolate retrieved content in the prompt.
* **Distance closer to 0:** _identical vectors_ / **Distance closer to 2:** _Opposing vectors_.
* https://weaviate.io/developers/weaviate/search/generative#single-prompt-search

In [25]:
prompt = "Create a historic 1 day trip itenerary \
    visiting only 3 specific places from the mentioned {destination} \
    considering the {cultural} values, \
    and prepare a dish for dinner using the mentioned {foods}"

response = collection.generate.near_text(
    query="Italy ancient history",
    single_prompt=prompt,
    return_properties=['destination','country','foods','cultural','description'],
    limit=2,    
    return_metadata=MetadataQuery(distance=True)
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.distance}")
    json_print(o.properties)      
    print(f"Azure OpenAI Response:\n------------------------ \n{word_wrap(o.generated)}\n------------------------\n")

In [39]:
prompt = "Translate to Spanish language the following values after the colon: \
    {destination}. {country}. {foods}. {cultural}. {description}"

response = collection.generate.near_text(
    query="Gaudi",
    single_prompt=prompt,
    return_properties=['destination','country','foods','cultural','description'],
    limit=1,
    #target_vector="title_vector",  # Specify the target vector for named vector collections    
    return_metadata=MetadataQuery(distance=True)
)

for i, o in enumerate(response.objects):    
    print(f"Rank {i + 1} - Score: {o.metadata.distance}")
    json_print(o.properties)      
    print(f"Azure OpenAI Response:\n------------------------ \n{word_wrap(o.generated)}\n------------------------\n")

### Grouped task search
* Grouped task search returns one response that includes all of the query results. 
* By default grouped task search uses all object properties in the prompt.
* https://weaviate.io/developers/weaviate/search/generative#grouped-task-search

In [ ]:
task = "Create a short summary of the provided data"

response = collection.generate.near_text(
    query="medieval castle",
    limit=3,
    grouped_task=task
)

print(f"Azure OpenAI Response:\n------------------------ \n{word_wrap(response.generated)}\n------------------------\n")

print("Weaviate Vector Search:")
for o in response.objects:
    print(o)